In [1]:
import random
import numpy as np

np.random.seed(1234)
random.seed(1234)

In [2]:
from numpy.random import *

import matplotlib.pyplot as pltj
import scipy.stats
from scipy.special import expit
import pandas as pd


In [3]:
num_data = 200

x_1 = randint(15,76, num_data)
x_2 = randint(0,2, num_data)

In [4]:
e_z = randn(num_data)
z_base = x_1 + (1-x_2)*10 - 40 + 5*e_z

z_prob = expit(0.1*z_base)
Z = np.array([])

for i in range(num_data):
    Z_i = np.random.choice(2, size=1, p=[1-z_prob[i], z_prob[i]])[0]
    Z = np.append(Z, Z_i)

In [5]:
e_y = randn(num_data)

Y = -x_1 + 30*x_2 + 10*Z + 80 + 10*e_y

In [6]:
df = pd.DataFrame({'年齢': x_1,
                   '性別': x_2,
                   'CMを見た':Z,
                   '購入量':Y,})

df.head()

,年齢,性別,CMを見た,購入量
0,62,0,1.0,24.464285
1,34,0,0.0,45.693411
2,53,1,1.0,64.998281
3,68,1,1.0,47.186898
4,27,1,0.0,100.114260


In [7]:
# 平均値を比べる

print(df[df['CMを見た']==1.0].mean())
print('-----')
print(df[df['CMを見た']==0.0].mean())

年齢       55.836066
性別        0.483607
CMを見た     1.000000
購入量      49.711478
dtype: float64
-----
年齢       32.141026
性別        0.692308
CMを見た     0.000000
購入量      68.827143
dtype: float64


## 傾向スコアの推定

In [8]:
# scikit-learnからロジスティック回帰をimport
from sklearn.linear_model import LogisticRegression

# 説明変数
X = df[['年齢','性別']]

# 非説明変数
Z = df['CMを見た']

# 回帰の実施
reg = LogisticRegression().fit(X,Z)

# 回帰した結果の係数を出力
print('係数beta:', reg.coef_)
print('係数alpha:', reg.intercept_)

係数beta: [[ 0.10562765 -1.38263933]]
係数alpha: [-3.37146523]


## 各人の傾向スコアを求める

In [9]:
Z_pre = reg.predict_proba(X)
print(Z_pre[0:5])
print('------')
print(Z[0:5])

[[0.04002323 0.95997677]
 [0.44525168 0.55474832]
 [0.30065918 0.69934082]
 [0.08101946 0.91898054]
 [0.87013558 0.12986442]]
------
0    1.0
1    0.0
2    1.0
3    1.0
4    0.0
Name: CMを見た, dtype: float64


## 平均処置効果ATEを求める

In [10]:
ATE_i = Y/Z_pre[:,1]*Z - Y/Z_pre[:,0]*(1-Z)
ATE = 1/len(Y)*ATE_i.sum()
print('推定したATE', ATE)

推定したATE 8.847476810855422
